# 🔍 数据库 N+1 查询问题详解

> **N+1 问题**是 ORM（对象关系映射）中最常见、最隐蔽、也最容易被忽视的性能杀手之一。
> 
> 本 Notebook 将通过实际案例，带你深入理解 N+1 问题的本质、产生原因、典型场景以及解决方案。

---

## 📚 目录

1. [什么是 N+1 问题？](#什么是-n1-问题)
2. [环境准备](#环境准备)
3. [N+1 问题的产生原因](#n1-问题的产生原因)
4. [典型场景演示](#典型场景演示)
5. [解决方案](#解决方案)
6. [最佳实践总结](#最佳实践总结)

## 🤔 什么是 N+1 问题？

想象一个场景：你要展示一个电商订单列表页面，每个订单需要显示下单客户的名字。

**你期望的查询方式：**
```sql
-- 1 条查询获取所有订单和客户信息
SELECT orders.*, customers.name FROM orders JOIN customers ON orders.customer_id = customers.id;
```

**实际发生的查询（N+1 问题）：**
```sql
-- 第 1 条：获取所有订单
SELECT * FROM orders;  

-- 第 2 条：获取第 1 个订单的客户
SELECT * FROM customers WHERE id = 1;

-- 第 3 条：获取第 2 个订单的客户
SELECT * FROM customers WHERE id = 2;

-- 第 4 条：获取第 3 个订单的客户
SELECT * FROM customers WHERE id = 3;

-- ... 如果有 N 个订单，就会执行 N+1 条查询！
```

### 📊 为什么叫 "N+1"？

| 组成部分 | 说明 |
|---------|------|
| **1** | 第一条查询，获取主表数据（如所有订单） |
| **N** | 对于主表的每一条记录，都额外执行一条查询获取关联数据 |

如果你有 100 个订单，就会产生 **101 条 SQL 查询**！这对数据库性能是灾难性的。

## 🛠️ 环境准备

首先，我们需要连接到测试数据库，并准备好模型定义。

In [1]:
# 环境配置（与 connect.ipynb 相同）
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

# 获取路径
notebook_path: Path = Path.cwd()
backend_dir: Path = notebook_path.parent.parent
root_dir: Path = backend_dir.parent

# 添加 backend 到 sys.path
if str(backend_dir) not in sys.path:
    sys.path.append(str(backend_dir))

# 加载测试环境变量
load_dotenv(root_dir / ".env.test", override=True)

print(f"✅ 环境: {os.getenv('ENVIRONMENT')}")
print(f"✅ 数据库: {os.getenv('POSTGRES_DB')}")

✅ 环境: test
✅ 数据库: db_test


In [2]:
# 创建数据库引擎（开启 SQL 日志，方便观察查询）
from app.core.config import settings
from sqlmodel import SQLModel, Session, create_engine, select, Field, Relationship
from sqlalchemy.engine import Engine
from sqlalchemy.orm import selectinload, joinedload
from sqlalchemy import func
from typing import Optional, List
from datetime import datetime, timezone, timedelta
from decimal import Decimal

# 🔑 关键：echo=True 会打印所有执行的 SQL 语句
engine: Engine = create_engine(str(settings.postgres_url), echo=True)

print(f"✅ 数据库连接成功")

✅ 数据库连接成功


In [3]:
# ============================================
# 🛒 电商订单系统 - 独立业务模型
# ============================================
# 表名都带有 nplusone_ 前缀，确保不与其他表冲突

def get_current_time() -> datetime:
    """返回当前时间"""
    return datetime.now(timezone(timedelta(hours=8))).replace(tzinfo=None)


class Customer(SQLModel, table=True):
    """客户表"""
    __tablename__ = "nplusone_customers"

    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(max_length=100, index=True)
    email: str = Field(max_length=200, unique=True)
    phone: Optional[str] = Field(default=None, max_length=20)

    # 关系：一个客户有多个订单
    orders: List["Order"] = Relationship(back_populates="customer")

    def __repr__(self) -> str:
        return f"Customer(id={self.id}, name='{self.name}')"
    __str__ = __repr__


class Product(SQLModel, table=True):
    """商品表"""
    __tablename__ = "nplusone_products"

    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(max_length=200, index=True)
    price: Decimal = Field(max_digits=10, decimal_places=2)
    stock: int = Field(default=0)
    category: str = Field(max_length=50)

    # 关系：一个商品可以出现在多个订单项中
    order_items: List["OrderItem"] = Relationship(back_populates="product")

    def __repr__(self) -> str:
        return f"Product(id={self.id}, name='{self.name}', price={self.price})"
    __str__ = __repr__


class Order(SQLModel, table=True):
    """订单表"""
    __tablename__ = "nplusone_orders"

    id: Optional[int] = Field(default=None, primary_key=True)
    order_no: str = Field(max_length=50, unique=True, index=True)
    status: str = Field(default="pending", max_length=20)  # pending, paid, shipped, completed
    total_amount: Decimal = Field(default=Decimal("0.00"), max_digits=12, decimal_places=2)
    created_at: datetime = Field(default_factory=get_current_time)

    # 外键
    customer_id: int = Field(foreign_key="nplusone_customers.id")

    # 关系
    customer: "Customer" = Relationship(back_populates="orders")
    items: List["OrderItem"] = Relationship(back_populates="order")

    def __repr__(self) -> str:
        return f"Order(id={self.id}, order_no='{self.order_no}', total={self.total_amount})"
    __str__ = __repr__


class OrderItem(SQLModel, table=True):
    """订单项表（订单明细）"""
    __tablename__ = "nplusone_order_items"

    id: Optional[int] = Field(default=None, primary_key=True)
    quantity: int = Field(default=1)
    unit_price: Decimal = Field(max_digits=10, decimal_places=2)
    subtotal: Decimal = Field(max_digits=12, decimal_places=2)

    # 外键
    order_id: int = Field(foreign_key="nplusone_orders.id")
    product_id: int = Field(foreign_key="nplusone_products.id")

    # 关系
    order: "Order" = Relationship(back_populates="items")
    product: "Product" = Relationship(back_populates="order_items")

    def __repr__(self) -> str:
        return f"OrderItem(id={self.id}, qty={self.quantity}, subtotal={self.subtotal})"
    __str__ = __repr__


# 创建表
SQLModel.metadata.create_all(engine)
print("✅ 电商订单系统表创建完成")
print("   - nplusone_customers (客户)")
print("   - nplusone_products (商品)")
print("   - nplusone_orders (订单)")
print("   - nplusone_order_items (订单项)")

2025-12-02 14:56:07,289 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-12-02 14:56:07,290 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-02 14:56:07,292 INFO sqlalchemy.engine.Engine select current_schema()
2025-12-02 14:56:07,294 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-02 14:56:07,295 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-12-02 14:56:07,296 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-12-02 14:56:07,298 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:56:07,305 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [4]:
# ============================================
# 📦 准备测试数据
# ============================================

with Session(engine) as session:
    # 检查是否已有数据
    existing_customers = session.exec(select(Customer)).all()

    if len(existing_customers) > 0:
        print(f"📊 数据已存在，跳过初始化")
        print(f"   - 客户: {len(existing_customers)} 位")
        products = session.exec(select(Product)).all()
        orders = session.exec(select(Order)).all()
        print(f"   - 商品: {len(products)} 件")
        print(f"   - 订单: {len(orders)} 个")
    else:
        print("🔄 初始化测试数据...")

        # 创建客户
        customers = [
            Customer(name="张三", email="zhangsan@shop.com", phone="13800001111"),
            Customer(name="李四", email="lisi@shop.com", phone="13800002222"),
            Customer(name="王五", email="wangwu@shop.com", phone="13800003333"),
            Customer(name="赵六", email="zhaoliu@shop.com", phone="13800004444"),
        ]
        session.add_all(customers)
        session.commit()
        for c in customers:
            session.refresh(c)

        # 创建商品
        products = [
            Product(name="iPhone 15 Pro", price=Decimal("8999.00"), stock=100, category="手机"),
            Product(name="MacBook Pro 14", price=Decimal("14999.00"), stock=50, category="电脑"),
            Product(name="AirPods Pro", price=Decimal("1899.00"), stock=200, category="配件"),
            Product(name="iPad Air", price=Decimal("4799.00"), stock=80, category="平板"),
            Product(name="Apple Watch", price=Decimal("2999.00"), stock=150, category="穿戴"),
            Product(name="Magic Keyboard", price=Decimal("699.00"), stock=300, category="配件"),
        ]
        session.add_all(products)
        session.commit()
        for p in products:
            session.refresh(p)

        # 创建订单和订单项
        import random
        order_count = 0
        for i, customer in enumerate(customers):
            # 每个客户 2-4 个订单
            for j in range(random.randint(2, 4)):
                order_count += 1
                order = Order(
                    order_no=f"ORD-2024-{order_count:04d}",
                    status=random.choice(["pending", "paid", "shipped", "completed"]),
                    customer_id=customer.id,
                    total_amount=Decimal("0.00")
                )
                session.add(order)
                session.commit()
                session.refresh(order)

                # 每个订单 1-4 个商品
                total = Decimal("0.00")
                selected_products = random.sample(products, random.randint(1, 4))
                for product in selected_products:
                    qty = random.randint(1, 3)
                    subtotal = product.price * qty
                    total += subtotal
                    item = OrderItem(
                        order_id=order.id,
                        product_id=product.id,
                        quantity=qty,
                        unit_price=product.price,
                        subtotal=subtotal
                    )
                    session.add(item)

                order.total_amount = total
                session.commit()

        print(f"✅ 测试数据创建完成")
        print(f"   - 客户: {len(customers)} 位")
        print(f"   - 商品: {len(products)} 件")
        print(f"   - 订单: {order_count} 个")

2025-12-02 14:56:23,304 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:56:23,316 INFO sqlalchemy.engine.Engine SELECT nplusone_customers.id, nplusone_customers.name, nplusone_customers.email, nplusone_customers.phone 
FROM nplusone_customers
2025-12-02 14:56:23,318 INFO sqlalchemy.engine.Engine [generated in 0.00120s] {}
📊 数据已存在，跳过初始化
   - 客户: 4 位
2025-12-02 14:56:23,323 INFO sqlalchemy.engine.Engine SELECT nplusone_products.id, nplusone_products.name, nplusone_products.price, nplusone_products.stock, nplusone_products.category 
FROM nplusone_products
2025-12-02 14:56:23,324 INFO sqlalchemy.engine.Engine [generated in 0.00082s] {}
2025-12-02 14:56:23,326 INFO sqlalchemy.engine.Engine SELECT nplusone_orders.id, nplusone_orders.order_no, nplusone_orders.status, nplusone_orders.total_amount, nplusone_orders.created_at, nplusone_orders.customer_id 
FROM nplusone_orders
2025-12-02 14:56:23,327 INFO sqlalchemy.engine.Engine [generated in 0.00064s] {}
   - 商品: 6 件
   - 订单: 13 个


## 🔥 N+1 问题的产生原因

N+1 问题的根本原因是 **ORM 的懒加载（Lazy Loading）机制**。

### 什么是懒加载？

当你访问一个对象的关联属性时，ORM 默认**不会**立即加载关联数据，而是等到你**真正访问**该属性时，才去数据库查询。

```python
order = session.get(Order, 1)       # 只查询 orders 表
print(order.customer.name)          # 这时才查询 customers 表！
```

### 懒加载的优缺点

| 优点 | 缺点 |
|------|------|
| 按需加载，节省初始查询时间 | 在循环中会产生 N+1 问题 |
| 代码简洁，无需显式 JOIN | 难以追踪实际执行的 SQL 数量 |
| 适合只访问部分关联数据的场景 | 网络延迟会被放大 N 倍 |

## 🎭 典型场景演示

### 场景一：订单列表显示客户信息

> 💡 最常见的业务需求：显示订单列表，每个订单需要显示下单客户的姓名

In [5]:
# ❌ N+1 问题演示：获取所有订单及客户信息
print("=" * 70)
print("🚨 N+1 问题演示 - 观察 SQL 查询数量")
print("=" * 70)

with Session(engine) as session:
    # 第 1 条查询：获取所有订单
    orders = session.exec(select(Order)).all()

    print(f"\n📋 共获取 {len(orders)} 个订单\n")
    print("-" * 70)

    # 遍历每个订单，访问客户信息
    # ⚠️ 每次访问 order.customer 都会触发一条额外的 SQL 查询！
    for order in orders:
        customer_name = order.customer.name  # 💥 这里触发懒加载！
        print(f"📦 {order.order_no} | 客户: {customer_name} | "
              f"金额: ¥{order.total_amount} | 状态: {order.status}")

print("\n" + "=" * 70)
print("⬆️ 观察上面的 SQL 日志：")
print(f"   1. 首先执行 1 条 SELECT orders 查询")
print(f"   2. 然后对每个订单都执行 1 条 SELECT customers 查询")
print(f"   3. 共 {len(orders) + 1} 条查询（N+1 问题！）")
print("=" * 70)

🚨 N+1 问题演示 - 观察 SQL 查询数量
2025-12-02 14:57:13,842 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:57:13,843 INFO sqlalchemy.engine.Engine SELECT nplusone_orders.id, nplusone_orders.order_no, nplusone_orders.status, nplusone_orders.total_amount, nplusone_orders.created_at, nplusone_orders.customer_id 
FROM nplusone_orders
2025-12-02 14:57:13,844 INFO sqlalchemy.engine.Engine [cached since 50.52s ago] {}

📋 共获取 13 个订单

----------------------------------------------------------------------
2025-12-02 14:57:13,848 INFO sqlalchemy.engine.Engine SELECT nplusone_customers.id AS nplusone_customers_id, nplusone_customers.name AS nplusone_customers_name, nplusone_customers.email AS nplusone_customers_email, nplusone_customers.phone AS nplusone_customers_phone 
FROM nplusone_customers 
WHERE nplusone_customers.id = %(pk_1)s
2025-12-02 14:57:13,849 INFO sqlalchemy.engine.Engine [generated in 0.00102s] {'pk_1': 1}
📦 ORD-2024-0001 | 客户: 张三 | 金额: ¥51794.00 | 状态: completed
📦 ORD-2024-0002 

### 场景二：订单详情（嵌套关联更严重）

> 💡 查看订单详情时，需要显示：
> - 订单信息
> - 客户信息
> - 订单中的所有商品信息
> 
> 这会产生更严重的 N+1 问题！

In [6]:
# ❌ 更严重的 N+1：订单 → 订单项 → 商品
print("=" * 70)
print("🚨 嵌套 N+1 问题 - 订单 → 客户 + 订单项 → 商品")
print("=" * 70)

with Session(engine) as session:
    # 只取前 3 个订单演示（否则日志太长）
    orders = session.exec(select(Order).limit(3)).all()

    query_count = 1  # 初始查询

    for order in orders:
        query_count += 1  # 获取客户信息
        print(f"\n📦 订单: {order.order_no}")
        print(f"   👤 客户: {order.customer.name} ({order.customer.phone})")
        print(f"   💰 总金额: ¥{order.total_amount}")
        print(f"   📝 商品明细:")

        query_count += 1  # 获取订单项列表
        for item in order.items:
            query_count += 1  # 获取商品信息
            print(f"      - {item.product.name} × {item.quantity} = ¥{item.subtotal}")

print(f"\n⚠️ 预估查询数: 约 {query_count} 条（实际可能更多）")
print("=" * 70)

🚨 嵌套 N+1 问题 - 订单 → 客户 + 订单项 → 商品
2025-12-02 14:57:24,634 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:57:24,636 INFO sqlalchemy.engine.Engine SELECT nplusone_orders.id, nplusone_orders.order_no, nplusone_orders.status, nplusone_orders.total_amount, nplusone_orders.created_at, nplusone_orders.customer_id 
FROM nplusone_orders 
 LIMIT %(param_1)s
2025-12-02 14:57:24,637 INFO sqlalchemy.engine.Engine [generated in 0.00108s] {'param_1': 3}

📦 订单: ORD-2024-0001
2025-12-02 14:57:24,640 INFO sqlalchemy.engine.Engine SELECT nplusone_customers.id AS nplusone_customers_id, nplusone_customers.name AS nplusone_customers_name, nplusone_customers.email AS nplusone_customers_email, nplusone_customers.phone AS nplusone_customers_phone 
FROM nplusone_customers 
WHERE nplusone_customers.id = %(pk_1)s
2025-12-02 14:57:24,641 INFO sqlalchemy.engine.Engine [cached since 10.79s ago] {'pk_1': 1}
   👤 客户: 张三 (13800001111)
   💰 总金额: ¥51794.00
   📝 商品明细:
2025-12-02 14:57:24,645 INFO sqlalchemy.e

### 场景三：API 返回订单列表（隐藏的 N+1）

> 💡 在实际 API 开发中，序列化响应数据时会不知不觉触发 N+1：

```python
# FastAPI 示例 - 这段代码隐藏着 N+1 问题！
@app.get("/api/orders")
def list_orders():
    orders = session.exec(select(Order)).all()
    return [
        {
            "order_no": order.order_no,
            "customer_name": order.customer.name,  # 💥 N+1！
            "item_count": len(order.items),        # 💥 又是 N+1！
            "total": order.total_amount
        }
        for order in orders
    ]
```

看起来代码很简洁，但每个订单都会触发 2 条额外查询！

---

## 💊 解决方案

解决 N+1 问题的核心思想是：**用更少的查询获取所需的所有数据**。

主要有以下几种策略：

| 策略 | 说明 | 适用场景 |
|------|------|----------|
| **Eager Loading（预加载）** | 在查询主表时，同时加载关联数据 | 确定需要关联数据时 |
| **selectinload** | 用 IN 子查询批量加载关联数据 | 一对多关系 |
| **joinedload** | 用 JOIN 一次性加载所有数据 | 多对一、一对一关系 |
| **手动 JOIN** | 自己写 JOIN 查询 | 需要精确控制时 |
| **子查询/聚合** | 用子查询计算统计值 | 只需要聚合数据时 |

### 方案一：使用 `joinedload`（JOIN 预加载）

> 💡 使用 SQL JOIN 在一条查询中获取订单和客户信息
> 
> 适合：**多对一** 关系（如 Order → Customer）

In [7]:
# ✅ 解决方案：使用 joinedload 预加载客户信息
print("=" * 70)
print("✅ 使用 joinedload 解决 N+1 - 只有 1 条 SQL！")
print("=" * 70)

with Session(engine) as session:
    # 🔑 关键：使用 options(joinedload(Order.customer)) 预加载
    stmt = select(Order).options(joinedload(Order.customer))
    orders = session.exec(stmt).unique().all()

    print(f"\n📋 共获取 {len(orders)} 个订单\n")
    print("-" * 70)

    # 现在访问 order.customer 不会触发额外查询！
    for order in orders:
        customer_name = order.customer.name  # ✅ 已预加载，无新查询
        print(f"📦 {order.order_no} | 客户: {customer_name} | "
              f"金额: ¥{order.total_amount} | 状态: {order.status}")

print("\n" + "=" * 70)
print("⬆️ 观察 SQL 日志：只有 1 条带 LEFT OUTER JOIN 的查询！")
print("=" * 70)

✅ 使用 joinedload 解决 N+1 - 只有 1 条 SQL！
2025-12-02 14:57:24,681 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:57:24,686 INFO sqlalchemy.engine.Engine SELECT nplusone_orders.id, nplusone_orders.order_no, nplusone_orders.status, nplusone_orders.total_amount, nplusone_orders.created_at, nplusone_orders.customer_id, nplusone_customers_1.id AS id_1, nplusone_customers_1.name, nplusone_customers_1.email, nplusone_customers_1.phone 
FROM nplusone_orders LEFT OUTER JOIN nplusone_customers AS nplusone_customers_1 ON nplusone_customers_1.id = nplusone_orders.customer_id
2025-12-02 14:57:24,687 INFO sqlalchemy.engine.Engine [generated in 0.00119s] {}

📋 共获取 13 个订单

----------------------------------------------------------------------
📦 ORD-2024-0001 | 客户: 张三 | 金额: ¥51794.00 | 状态: completed
📦 ORD-2024-0002 | 客户: 张三 | 金额: ¥71992.00 | 状态: pending
📦 ORD-2024-0003 | 客户: 张三 | 金额: ¥23693.00 | 状态: paid
📦 ORD-2024-0004 | 客户: 张三 | 金额: ¥67291.00 | 状态: completed
📦 ORD-2024-0005 | 客户: 李四 | 金额: ¥3

### 方案二：使用 `selectinload`（IN 子查询预加载）

> 💡 先查询主表，然后用 `WHERE id IN (...)` 批量加载关联数据
> 
> 适合：**一对多** 关系（如 Order → OrderItems）
> 
> 会产生 2 条查询，但比 N+1 好多了！

In [8]:
# ✅ selectinload - 适合一对多关系（订单 → 订单项）
print("=" * 70)
print("✅ 使用 selectinload 加载订单项 - 只有 2 条 SQL！")
print("=" * 70)

with Session(engine) as session:
    # selectinload 使用 IN 查询批量加载关联数据
    stmt = select(Order).options(selectinload(Order.items))
    orders = session.exec(stmt).all()

    print(f"\n📋 共获取 {len(orders)} 个订单\n")
    print("-" * 70)

    for order in orders:
        item_count = len(order.items)  # ✅ 已预加载
        print(f"📦 {order.order_no} | 商品数量: {item_count} 种 | "
              f"总金额: ¥{order.total_amount}")

print("\n" + "=" * 70)
print("⬆️ 观察：1条查订单 + 1条用 IN 批量查订单项 = 2条 SQL")
print("=" * 70)

✅ 使用 selectinload 加载订单项 - 只有 2 条 SQL！
2025-12-02 14:57:24,706 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:57:24,708 INFO sqlalchemy.engine.Engine SELECT nplusone_orders.id, nplusone_orders.order_no, nplusone_orders.status, nplusone_orders.total_amount, nplusone_orders.created_at, nplusone_orders.customer_id 
FROM nplusone_orders
2025-12-02 14:57:24,710 INFO sqlalchemy.engine.Engine [generated in 0.00150s] {}
2025-12-02 14:57:24,714 INFO sqlalchemy.engine.Engine SELECT nplusone_order_items.order_id AS nplusone_order_items_order_id, nplusone_order_items.id AS nplusone_order_items_id, nplusone_order_items.quantity AS nplusone_order_items_quantity, nplusone_order_items.unit_price AS nplusone_order_items_unit_price, nplusone_order_items.subtotal AS nplusone_order_items_subtotal, nplusone_order_items.product_id AS nplusone_order_items_product_id 
FROM nplusone_order_items 
WHERE nplusone_order_items.order_id IN (%(primary_keys_1)s, %(primary_keys_2)s, %(primary_keys_3)s, %(p

### 方案三：嵌套预加载（解决深层 N+1）

> 💡 对于多层嵌套关系，可以链式使用预加载

In [9]:
# ✅ 嵌套预加载：订单 → 客户 + 订单项 → 商品
print("=" * 70)
print("✅ 嵌套 selectinload - 加载完整订单详情！")
print("=" * 70)

with Session(engine) as session:
    stmt = select(Order).options(
        joinedload(Order.customer),                      # 加载客户
        selectinload(Order.items).joinedload(OrderItem.product)  # 加载订单项和商品
    )
    orders = session.exec(stmt).unique().all()

    print(f"\n📋 订单详情报表\n")

    for order in orders[:3]:  # 只显示前3个订单
        print("=" * 60)
        print(f"📦 订单号: {order.order_no}")
        print(f"👤 客户: {order.customer.name} ({order.customer.email})")
        print(f"📅 下单时间: {order.created_at}")
        print(f"📊 状态: {order.status}")
        print("-" * 60)

        for item in order.items:
            product = item.product  # ✅ 已预加载
            subtotal = item.quantity * item.unit_price
            print(f"   📱 {product.name} | 单价: ¥{item.unit_price} × {item.quantity} = ¥{subtotal}")

        print(f"   {'─' * 50}")
        print(f"   💰 订单总额: ¥{order.total_amount}")
        print()

print("=" * 70)
print("⬆️ 高效：只需 3-4 条 SQL 完成所有数据加载！")
print("=" * 70)

✅ 嵌套 selectinload - 加载完整订单详情！
2025-12-02 14:57:24,740 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:57:24,743 INFO sqlalchemy.engine.Engine SELECT nplusone_orders.id, nplusone_orders.order_no, nplusone_orders.status, nplusone_orders.total_amount, nplusone_orders.created_at, nplusone_orders.customer_id, nplusone_customers_1.id AS id_1, nplusone_customers_1.name, nplusone_customers_1.email, nplusone_customers_1.phone 
FROM nplusone_orders LEFT OUTER JOIN nplusone_customers AS nplusone_customers_1 ON nplusone_customers_1.id = nplusone_orders.customer_id
2025-12-02 14:57:24,744 INFO sqlalchemy.engine.Engine [generated in 0.00094s] {}
2025-12-02 14:57:24,750 INFO sqlalchemy.engine.Engine SELECT nplusone_order_items.order_id AS nplusone_order_items_order_id, nplusone_order_items.id AS nplusone_order_items_id, nplusone_order_items.quantity AS nplusone_order_items_quantity, nplusone_order_items.unit_price AS nplusone_order_items_unit_price, nplusone_order_items.subtotal AS nplus

### 方案四：手动 JOIN（最精确控制）

> 💡 当你只需要部分字段，或需要复杂的过滤条件时

In [10]:
# ✅ 手动 JOIN - 灵活控制返回字段
print("=" * 70)
print("✅ 手动 JOIN 获取客户订单统计")
print("=" * 70)

with Session(engine) as session:
    # 使用 func.count 和 func.sum 进行聚合
    from sqlmodel import func

    stmt = (
        select(
            Customer.name,
            Customer.email,
            Customer.phone,
            func.count(Order.id).label("order_count"),
            func.sum(Order.total_amount).label("total_spent")
        )
        .join(Order, Order.customer_id == Customer.id)
        .group_by(Customer.id, Customer.name, Customer.email, Customer.phone)
        .order_by(func.sum(Order.total_amount).desc())
    )

    results = session.exec(stmt).all()

    print(f"\n👥 客户消费排行榜\n")
    print("-" * 70)
    print(f"{'排名':<4} {'客户':<12} {'联系电话':<15} {'订单数':<8} {'消费总额':<12}")
    print("-" * 70)

    for rank, (name, email, phone, order_count, total_spent) in enumerate(results, 1):
        phone_display = phone or "未提供"
        print(f"{rank:<4} 👤 {name:<10} {phone_display:<15} {order_count:<8} ¥{total_spent or 0:,.2f}")

print("\n" + "=" * 70)
print("⬆️ 手动 JOIN：精确控制查询，只返回需要的字段")
print("=" * 70)

✅ 手动 JOIN 获取客户订单统计
2025-12-02 14:57:24,777 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:57:24,785 INFO sqlalchemy.engine.Engine SELECT nplusone_customers.name, nplusone_customers.email, nplusone_customers.phone, count(nplusone_orders.id) AS order_count, sum(nplusone_orders.total_amount) AS total_spent 
FROM nplusone_customers JOIN nplusone_orders ON nplusone_orders.customer_id = nplusone_customers.id GROUP BY nplusone_customers.id, nplusone_customers.name, nplusone_customers.email, nplusone_customers.phone ORDER BY sum(nplusone_orders.total_amount) DESC
2025-12-02 14:57:24,786 INFO sqlalchemy.engine.Engine [generated in 0.00208s] {}

👥 客户消费排行榜

----------------------------------------------------------------------
排名   客户           联系电话            订单数      消费总额        
----------------------------------------------------------------------
1    👤 张三         13800001111     4        ¥214,770.00
2    👤 李四         13800002222     4        ¥196,366.00
3    👤 王五         13800

### 方案五：使用子查询计算聚合值

> 💡 如果你只需要统计数据（如销售数量），不需要加载所有订单项

In [11]:
# ✅ 子查询聚合 - 获取每个商品的销售统计
print("=" * 70)
print("✅ 子查询聚合 - 商品销售统计")
print("=" * 70)

with Session(engine) as session:
    from sqlmodel import func

    # 子查询：计算每个商品的销售数量和销售额
    subquery = (
        select(
            OrderItem.product_id,
            func.sum(OrderItem.quantity).label("sold_qty"),
            func.sum(OrderItem.quantity * OrderItem.unit_price).label("revenue")
        )
        .group_by(OrderItem.product_id)
        .subquery()
    )

    # 主查询：关联商品表
    stmt = (
        select(
            Product.name,
            Product.category,
            Product.price,
            Product.stock,
            subquery.c.sold_qty,
            subquery.c.revenue
        )
        .outerjoin(subquery, Product.id == subquery.c.product_id)
        .order_by(subquery.c.revenue.desc().nullslast())
    )

    results = session.exec(stmt).all()

    print(f"\n📊 商品销售报表\n")
    print("-" * 80)
    print(f"{'商品名称':<15} {'分类':<10} {'单价':<10} {'库存':<6} {'已售':<6} {'销售额':<12}")
    print("-" * 80)

    for name, category, price, stock, sold_qty, revenue in results:
        sold = sold_qty or 0
        rev = revenue or Decimal('0')
        status = "🔥" if sold > 3 else "📦"
        print(f"{status} {name:<13} {category:<10} ¥{price:<8} {stock:<6} {sold:<6} ¥{rev:,.2f}")

print("\n" + "=" * 70)
print("⬆️ 子查询：适合复杂聚合，减少主查询数据量")
print("=" * 70)

✅ 子查询聚合 - 商品销售统计
2025-12-02 14:57:24,807 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:57:24,813 INFO sqlalchemy.engine.Engine SELECT nplusone_products.name, nplusone_products.category, nplusone_products.price, nplusone_products.stock, anon_1.sold_qty, anon_1.revenue 
FROM nplusone_products LEFT OUTER JOIN (SELECT nplusone_order_items.product_id AS product_id, sum(nplusone_order_items.quantity) AS sold_qty, sum(nplusone_order_items.quantity * nplusone_order_items.unit_price) AS revenue 
FROM nplusone_order_items GROUP BY nplusone_order_items.product_id) AS anon_1 ON nplusone_products.id = anon_1.product_id ORDER BY anon_1.revenue DESC NULLS LAST
2025-12-02 14:57:24,814 INFO sqlalchemy.engine.Engine [generated in 0.00088s] {}

📊 商品销售报表

--------------------------------------------------------------------------------
商品名称            分类         单价         库存     已售     销售额         
--------------------------------------------------------------------------------
🔥 MacBook Pr

---

## 📊 性能对比

让我们用代码实际对比 N+1 问题和各种优化方案的查询次数：

In [12]:
# 📊 性能对比：统计各方案的 SQL 查询次数
import time
from sqlalchemy import event
from sqlalchemy.engine import Engine

# 创建一个查询计数器
class QueryCounter:
    def __init__(self):
        self.count = 0
        self.queries = []

    def reset(self):
        self.count = 0
        self.queries = []

    def record(self, query):
        self.count += 1
        # 只记录 SELECT 语句（过滤掉 BEGIN/COMMIT/ROLLBACK）
        if query.strip().upper().startswith("SELECT"):
            self.queries.append(query[:80] + "..." if len(query) > 80 else query)

counter = QueryCounter()

# 注册事件监听器
@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(conn, cursor, statement, parameters, context, executemany):
    counter.record(statement)

print("✅ 查询计数器已设置")
print("   接下来的对比测试会统计每种方案的 SQL 查询次数")

✅ 查询计数器已设置
   接下来的对比测试会统计每种方案的 SQL 查询次数


In [13]:
# 🔬 实际性能对比测试
import time

results = []

# ❌ 方案0：N+1 问题（默认懒加载）
counter.reset()
start = time.perf_counter()
with Session(engine) as session:
    orders = session.exec(select(Order)).all()
    for order in orders:
        _ = order.customer.name  # 触发懒加载
        for item in order.items:  # 触发懒加载
            _ = item.product.name  # 触发懒加载
elapsed = (time.perf_counter() - start) * 1000
results.append(("❌ N+1 懒加载", counter.count, f"{elapsed:.2f}ms"))

# ✅ 方案1：joinedload（多对一）
counter.reset()
start = time.perf_counter()
with Session(engine) as session:
    stmt = select(Order).options(joinedload(Order.customer))
    orders = session.exec(stmt).unique().all()
    for order in orders:
        _ = order.customer.name
elapsed = (time.perf_counter() - start) * 1000
results.append(("✅ joinedload", counter.count, f"{elapsed:.2f}ms"))

# ✅ 方案2：selectinload（一对多）
counter.reset()
start = time.perf_counter()
with Session(engine) as session:
    stmt = select(Order).options(selectinload(Order.items))
    orders = session.exec(stmt).all()
    for order in orders:
        _ = len(order.items)
elapsed = (time.perf_counter() - start) * 1000
results.append(("✅ selectinload", counter.count, f"{elapsed:.2f}ms"))

# ✅ 方案3：嵌套预加载（完整订单详情）
counter.reset()
start = time.perf_counter()
with Session(engine) as session:
    stmt = select(Order).options(
        joinedload(Order.customer),
        selectinload(Order.items).joinedload(OrderItem.product)
    )
    orders = session.exec(stmt).unique().all()
    for order in orders:
        _ = order.customer.name
        for item in order.items:
            _ = item.product.name
elapsed = (time.perf_counter() - start) * 1000
results.append(("✅ 嵌套预加载", counter.count, f"{elapsed:.2f}ms"))

# ✅ 方案4：手动 JOIN 聚合
counter.reset()
start = time.perf_counter()
with Session(engine) as session:
    from sqlmodel import func
    stmt = (
        select(Customer.name, func.count(Order.id), func.sum(Order.total_amount))
        .join(Order, Order.customer_id == Customer.id)
        .group_by(Customer.id, Customer.name)
    )
    _ = session.exec(stmt).all()
elapsed = (time.perf_counter() - start) * 1000
results.append(("✅ 手动JOIN聚合", counter.count, f"{elapsed:.2f}ms"))

# 打印对比结果
print("=" * 70)
print("📊 性能对比结果（订单数量: 13）")
print("=" * 70)
print(f"\n{'方案':<20} {'SQL查询次数':<15} {'耗时':<15} {'效率提升':<15}")
print("-" * 70)

baseline = results[0][1]  # N+1 的查询次数作为基准
for name, count, elapsed in results:
    if count == baseline:
        improvement = "基准"
    else:
        improvement = f"减少 {baseline - count} 次 ({(1 - count/baseline)*100:.0f}%)"
    print(f"{name:<20} {count:<15} {elapsed:<15} {improvement}")

print("\n" + "=" * 70)
print("💡 结论：预加载可以将查询次数从 N+1 减少到常数级别！")
print("=" * 70)

2025-12-02 14:57:24,848 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 14:57:24,849 INFO sqlalchemy.engine.Engine SELECT nplusone_orders.id, nplusone_orders.order_no, nplusone_orders.status, nplusone_orders.total_amount, nplusone_orders.created_at, nplusone_orders.customer_id 
FROM nplusone_orders
2025-12-02 14:57:24,850 INFO sqlalchemy.engine.Engine [cached since 61.52s ago] {}
2025-12-02 14:57:24,853 INFO sqlalchemy.engine.Engine SELECT nplusone_customers.id AS nplusone_customers_id, nplusone_customers.name AS nplusone_customers_name, nplusone_customers.email AS nplusone_customers_email, nplusone_customers.phone AS nplusone_customers_phone 
FROM nplusone_customers 
WHERE nplusone_customers.id = %(pk_1)s
2025-12-02 14:57:24,854 INFO sqlalchemy.engine.Engine [cached since 11.01s ago] {'pk_1': 1}
2025-12-02 14:57:24,858 INFO sqlalchemy.engine.Engine SELECT nplusone_order_items.id AS nplusone_order_items_id, nplusone_order_items.quantity AS nplusone_order_items_quantity, nplus

In [14]:
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                        🎯 N+1 查询问题解决方案总结                           ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  📌 什么是 N+1 问题？                                                        ║
║  ────────────────────────────────────────────────────────────────────────   ║
║  加载 N 条主记录后，访问每条记录的关联数据时各触发 1 次查询                   ║
║  总共执行 1 + N 次数据库查询，严重影响性能                                   ║
║                                                                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  🔧 解决方案对比（以 100 个订单为例）                                        ║
║  ────────────────────────────────────────────────────────────────────────   ║
║                                                                              ║
║  方案              │ SQL数量 │ 适用场景            │ 特点                   ║
║  ─────────────────┼────────┼────────────────────┼──────────────────────  ║
║  ❌ 默认懒加载      │ 101     │ 小数据量           │ N+1 问题，性能差        ║
║  ✅ joinedload     │ 1       │ 多对一（订单→客户） │ 单条 JOIN，数据可能重复 ║
║  ✅ selectinload   │ 2       │ 一对多（订单→订单项）│ IN 查询，更少内存占用  ║
║  ✅ 手动 JOIN      │ 1       │ 需要精确控制字段    │ 灵活，适合聚合统计     ║
║  ✅ 子查询聚合     │ 1       │ 复杂聚合统计        │ 先聚合后关联，高效     ║
║                                                                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  💡 最佳实践                                                                 ║
║  ────────────────────────────────────────────────────────────────────────   ║
║  1. 开发时启用 echo=True 观察 SQL 语句                                       ║
║  2. 多对一关系优先使用 joinedload                                            ║
║  3. 一对多关系优先使用 selectinload                                          ║
║  4. 嵌套关系使用链式加载：selectinload(...).joinedload(...)                  ║
║  5. 只需要统计值时使用 func.count/sum 聚合                                   ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")


╔══════════════════════════════════════════════════════════════════════════════╗
║                        🎯 N+1 查询问题解决方案总结                           ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  📌 什么是 N+1 问题？                                                        ║
║  ────────────────────────────────────────────────────────────────────────   ║
║  加载 N 条主记录后，访问每条记录的关联数据时各触发 1 次查询                   ║
║  总共执行 1 + N 次数据库查询，严重影响性能                                   ║
║                                                                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  🔧 解决方案对比（以 100 个订单为例）                                        ║
║  ────────────────────────────────────────────────────────────────────────   ║
║                      

---

## 📝 最佳实践总结

### 🎯 选择正确的预加载策略

| 关系类型 | 推荐策略 | 原因 |
|---------|---------|------|
| **多对一** (Order → Customer) | `joinedload` | 数据量小，一条 JOIN 即可 |
| **一对一** | `joinedload` | 同上 |
| **一对多** (Order → OrderItems) | `selectinload` | 避免笛卡尔积，用 IN 查询 |
| **多对多** | `selectinload` | 同上 |

### 🛡️ 防止 N+1 的代码规范

```python
# ❌ 危险：在循环中访问关联属性
orders = session.exec(select(Order)).all()
for order in orders:
    print(order.customer.name)  # 💥 N+1!

# ✅ 安全：预加载关联数据
orders = session.exec(
    select(Order).options(joinedload(Order.customer))
).unique().all()
for order in orders:
    print(order.customer.name)  # ✅ 已预加载
```

### 🔧 调试技巧

1. **开启 SQL 日志**：`create_engine(..., echo=True)`
2. **使用 SQL 分析工具**：如 `sqlalchemy-query-count`
3. **监控数据库查询**：使用 APM 工具（如 New Relic、Datadog）

### ⚠️ 注意事项

1. **不要过度预加载**：如果不需要关联数据，不要预加载
2. **注意笛卡尔积**：多个一对多关系用 `joinedload` 会产生大量重复数据
3. **unique() 的使用**：使用 `joinedload` 后，记得调用 `.unique()` 去重

---

## 🎓 总结

| 要点 | 说明 |
|------|------|
| **什么是 N+1** | 查询 N 条主数据时，额外产生 N 条关联查询 |
| **产生原因** | ORM 的懒加载机制 + 在循环中访问关联属性 |
| **危害** | 数据库负载暴增，响应时间变慢，严重时会拖垮系统 |
| **解决方案** | 预加载（joinedload/selectinload）、手动 JOIN、子查询 |
| **最佳实践** | 明确需要的数据，选择合适的加载策略 |

> 💡 **记住**：N+1 问题是隐蔽的性能杀手，在开发阶段就应该注意。
> 
> 开启 SQL 日志（`echo=True`）是发现 N+1 问题的最简单方法！

---

*Happy Coding! 🚀*